In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D

from tensorflow.keras.callbacks import TensorBoard
import pickle
import time



In [2]:
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.4)
sess = tf.Session(config=tf.ConfigProto(gpu_options = gpu_options))

In [3]:
pickle_in = open("X.pickle","rb")
X = pickle.load(pickle_in)

pickle_in = open("y.pickle","rb")
y = pickle.load(pickle_in)

X = X/255.0



In [4]:
dense_layers = [0,1,2]
layer_sizes = [50, 100, 128]
conv_layers = [5,4,3]

# the best model params that i found so far (0, 50, 5)


In [5]:
for dense_layer in dense_layers:
    for layer_size in layer_sizes:
        for conv_layer in conv_layers:
            NAME = "{}-conv-{}-nodes-{}-dense-{}".format(conv_layer, layer_size, dense_layer, int(time.time()))
            print(NAME)

            model = Sequential()

            model.add(Conv2D(layer_size, (3, 3), input_shape=X.shape[1:]))
            model.add(Activation('relu'))
            model.add(MaxPooling2D(pool_size=(2, 2)))

            for l in range(conv_layer-1):
                model.add(Conv2D(layer_size, (3, 3)))
                model.add(Activation('relu'))
                model.add(MaxPooling2D(pool_size=(2, 2)))

            model.add(Flatten())

            for _ in range(dense_layer):
                model.add(Dense(layer_size))
                model.add(Activation('relu'))

            model.add(Dense(1))
            model.add(Activation('sigmoid'))

            tensorboard = TensorBoard(log_dir="logs/{}".format(NAME))

            model.compile(loss='binary_crossentropy',
                          optimizer='adam',
                          metrics=['accuracy'],
                          )

            model.fit(X, y,
                      batch_size=32,
                      epochs=9,    # more than 9 epochs will cause overfitting in this case
                      validation_split=0.3,
                      callbacks=[tensorboard])

5-conv-50-nodes-0-dense-1563799070
Instructions for updating:
Colocations handled automatically by placer.
Train on 17462 samples, validate on 7484 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/9
17462/17462 [==============================] - 44s 3ms/sample - loss: 0.6774 - acc: 0.5575 - val_loss: 0.6620 - val_acc: 0.5925
Epoch 2/9
17462/17462 [==============================] - 22s 1ms/sample - loss: 0.5742 - acc: 0.6976 - val_loss: 0.5016 - val_acc: 0.7682
Epoch 3/9
17462/17462 [==============================] - 21s 1ms/sample - loss: 0.4715 - acc: 0.7773 - val_loss: 0.4267 - val_acc: 0.8036
Epoch 4/9
17462/17462 [==============================] - 21s 1ms/sample - loss: 0.4000 - acc: 0.8153 - val_loss: 0.3820 - val_acc: 0.8307
Epoch 5/9
17462/17462 [==============================] - 21s 1ms/sample - loss: 0.3417 - acc: 0.8484 - val_loss: 0.3557 - val_acc: 0.8399
Epoch 6/9
17462/17462 [==============================] - 21s 1ms/sample - loss: 0.2996 - acc: 0.8689 - val

## to access tensorboard ==> at cmd type => tensorboard --logdir==training:logs/ --host=127.0.0.1

In [6]:
model.save('503')

In [8]:
mdl = tf.keras.models.load_model('50x3')
